In [1]:
import pandas as pd
import sqlite3

# 9.1 Lendo dados de bancos de dados SQL
Até agora, falamos apenas sobre a leitura de dados de arquivos CSV. Essa é uma forma bastante comum de armazenar dados, mas existem muitas outras! Os pandas podem ler HTML, JSON, SQL, Excel (!!!), HDF5, Stata e algumas outras coisas. Neste capítulo, falaremos sobre a leitura de dados de bancos de dados SQL.

Você pode ler dados de um banco de dados SQL usando a `pd.read_sql`. A função `read_sql` converterá automaticamente nomes de colunas SQL em nomes de colunas DataFrame.

`read_sql` recebe 2 argumentos: uma `SELECT` instrução e um objeto de conexão com o banco de dados. Isso é ótimo porque significa que você pode ler qualquer tipo de banco de dados SQL -- não importa se é MySQL, SQLite, PostgreSQL ou qualquer outro.

Este exemplo lê de um banco de dados SQLite, mas qualquer outro banco de dados funcionaria da mesma maneira.

In [2]:
con = sqlite3.connect('pandas-cookbook-master/data/weather_2012.sqlite')
df = pd.read_sql('SELECT * from weather_2012 LIMIT 3', con)
df

,id,date_time,temp
0,1,2012-01-01 00:00:00,-1.8
1,2,2012-01-01 01:00:00,-1.8
2,3,2012-01-01 02:00:00,-1.8


`read_sql` não define automaticamente a chave primária (`id`) para ser o índice do dataframe. Você pode fazer isso adicionando um `index_col` argumento a `read_sql`.

Se você já usou `read_csv` muito, já deve ter visto que tem `index_col` argumento também. Este se comporta da mesma maneira.

In [3]:
df = pd.read_sql('SELECT * from weather_2012 LIMIT 3', con, index_col = 'id')
df

,date_time,temp
id,,
1,2012-01-01 00:00:00,-1.8
2,2012-01-01 01:00:00,-1.8
3,2012-01-01 02:00:00,-1.8


Se você deseja que seu dataframe seja indexado por mais de uma coluna, pode fornecer uma lista de colunas para `index_col`:

In [4]:
df = pd.read_sql("SELECT * from weather_2012 LIMIT 3", con, 
                 index_col=['id', 'date_time'])
df

,,temp
id,date_time,
1,2012-01-01 00:00:00,-1.8
2,2012-01-01 01:00:00,-1.8
3,2012-01-01 02:00:00,-1.8


## 9.2 Gravando em um banco de dados SQLite
Pandas tem uma `write_frame` função que cria uma tabela de banco de dados a partir de um dataframe. No momento, isso funciona apenas para bancos de dados SQLite. Vamos usá-lo para mover nossos dados meteorológicos de 2012 para o SQL.

Você notará que esta função está em `pd.io.sql`. Há uma tonelada de funções úteis para ler e gravar vários tipos de dados no `pd.io`, e vale a pena gastar algum tempo explorando-as.

In [8]:
# NÃO VAI FUNCIONAR 
# NÃO HÁ SQL NO MEU COMPUTADOR... LOGO NÃO HÁ UM 'test_db,sqlite'.
weather_df = pd.read_csv('pandas-cookbook-master/data/weather_2012.csv')
con = sqlite3.connect("../data/test_db.sqlite")
con.execute("DROP TABLE IF EXISTS weather_2012")
weather_df.to_sql("weather_2012", con)

OperationalError: unable to open database file

In [11]:
df = pd.read_sql('SELECT * from weather_2012', con, index_col = 'id')
df

,date_time,temp
id,,
1,2012-01-01 00:00:00,-1.8
2,2012-01-01 01:00:00,-1.8
3,2012-01-01 02:00:00,-1.8
4,2012-01-01 03:00:00,-1.5
5,2012-01-01 04:00:00,-1.5
...,...,...
96,2012-01-04 23:00:00,-9.6
97,2012-01-05 00:00:00,-8.8
98,2012-01-05 01:00:00,-7.5
